In [1]:
import ctypes
import multiprocessing
import numpy as np
import os
import time
from pynq.lib.pmod import Pmod_IO
from pynq.lib import Pmod_PWM

In [2]:
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")

In [3]:
msg = 0b000000
print(msg)
msg = msg << 1
msg |= (1 & 1)
msg &= 0x3F
print(msg)
msg = msg << 1
msg |= (1 & 1)
msg &= 0x3F
print(msg)
if msg == 0b000011:
    print(True)
else:
    print(False)
msg = 0b000000
print(msg)

0
1
3
True
0


In [3]:
def transceiver (tx_flag, tx_buf, rx_counter, rx_buf, quit_flag):
    period=0.05
    inpin = Pmod_IO (base.PMODA, 3, 'in')
    outpwm = Pmod_PWM(base.PMODB, 3)
    start_time=time.perf_counter()
    timecounter=start_time
    strtMsg = 0b000000
    msgStarted = False
    
    i=0
    
    while not quit_flag.value:
        # Do TX stuff - TODO: read from tx buf
#         if i % 2 == 0:
#             outpwm.generate(26,50)
#         else:
#             outpwm.stop()
        # Do RX stuff - TODO: properly index
        if msgStarted == True:
            rx_buf[0][i] = (1-inpin.read())
            i += 1
            if i == 199:
                msgStarted = False
                i = 0
                print("Message Captured")
                break
            
        if msgStarted == False:
            val = (1-inpin.read())
            strtMsg = strtMsg << 1
            strtMsg |= (val & 1)
            strtMsg &= 0x3F
            if strtMsg == 0b101101:
                msgStarted = True
                strtMsg = 0b000000
     

        timecounter += period 
        now=time.perf_counter()
        if now < (timecounter+period):
            time.sleep(timecounter+period-now)

            
def data_processor (tx_flag, tx_buf, rx_counter, rx_bufs, quit_flag):
    j = 0
    while not quit_flag.value:
        # Do data stuff
        time.sleep(0.25)
        j += 1
        
def button_handler (quit_flag):
    btns = base.btns_gpio
    while not btns[0].read():
        time.sleep(0.25)
    quit_flag.value = True

In [4]:
rx_buffers=[]
for i in range(3):
    # TODO: determine if we need the lock
    rx_buffers.append(multiprocessing.Array(ctypes.c_bool, 200,lock=False))

rx_counter=multiprocessing.Value(ctypes.c_uint8)
rx_counter.value=0
    
# TODO: update to message size. Do we need multiple?
tx_buffer=multiprocessing.Array(ctypes.c_uint16,128)

tx_flag=multiprocessing.Value(ctypes.c_bool)
tx_flag.value=False

quit_flag=multiprocessing.Value(ctypes.c_bool, lock=False)
quit_flag.value=False

p_xcvr = multiprocessing.Process(target=transceiver, args=(tx_flag,tx_buffer,rx_counter,rx_buffers,quit_flag))

p_data = multiprocessing.Process(target=data_processor, args=(tx_flag,tx_buffer,rx_counter,rx_buffers,quit_flag))

p_btn = multiprocessing.Process(target=button_handler, args=[quit_flag])

p_xcvr.start() # start the process
os.system("taskset -p -c {} {} > /dev/null".format(1, p_xcvr.pid))
p_data.start() # start the process
os.system("taskset -p -c {} {} > /dev/null".format(0, p_data.pid))
p_btn.start()
os.system("taskset -p -c {} {} > /dev/null".format(0, p_btn.pid))

print("Started processes")

p_xcvr.join()
p_data.join()
p_btn.join()

print("Quit")
print([val for val in rx_buffers[0]])
# for arr in rx_buffers:
#     print([val for val in arr])

Started processes
Message Captured
Quit
[True, False, True, True, True, False, True, False, True, True, True, False, True, False, True, True, True, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False,

In [6]:
print([val for val in rx_buffers[0]])



[False, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, Fals